# Value Lost

A notebook to determine the value lost when applying a maintenance strategy defined in an asset model to a population

Author: gavin.treseder@essentialenergy.com.au

## Current Working Directory
Set the file path so that we can import pof code and load files

In [31]:
#  Add root folder to python path
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

## Import packages

Import the packages that are required for this process 

In [32]:
import copy

from IPython.display import clear_output
import numpy as np 
import pandas as pd
import plotly.express as px
import scipy.stats as ss


from pof import Component
from pof.interface import figures
from pof.loader.asset_model_loader import AssetModelLoader
from pof.paths import Paths

## Load the Asset Model

In [33]:
asset_model_filename = "Asset Model - Pole - Timber.xlsx"

paths = Paths()
model_path = paths.demo_path + os.sep + asset_model_filename

if asset_model_filename is not None:
    
    aml = AssetModelLoader()
    comp_data = aml.load(model_path)
    comp = Component.from_dict(comp_data["pole"])

    print("Asset Model loaded")
    pass
else:
    comp = Component.demo()
    print("Demo Model loaded")

Asset Model loaded


In [ ]:
consequence_range = np.arange(0, 2e6, 1e5)
t_interval_range = np.arrange(0, 10, 0.25)
n_iterations = 500

for consequence in consequence_range:
    for t_interval in t_interval_range:




In [29]:
comp.units = 'months'

In [30]:
consequence_range = np.arange(0, 2e6, 1e5)

var_id = 'pole-task_group_name-groundline-t_interval'
lower = 0 * 12
upper = 10 * 12
t_end = 100 * 12
n_iterations = 100
step_size = 0.25 * 12

sens_results = dict()

for consequence_cost in consequence_range:

    # Update 
    comp.update('consequence-cost', consequence_cost)

    # Simulate
    comp.expected_sensitivity(
        var_id=var_id,
        lower=lower,
        upper=upper,
        t_end=t_end,
        step_size=step_size,
        n_iterations=n_iterations,
    )

    # Report
    df_sens = copy.copy(comp.df_sens)
    df_sens['consequence-cost'] = consequence_cost
    sens_results['consequence-cost'] = df_sens

df_results = pd.DataFrame.from_dict(sens_results)



100%|██████████| 100/100 [02:53<00:00,  1.73s/it]
ERROR:root:Error at %s
Traceback (most recent call last):
  File "C:\Users\gtreseder\OneDrive - KPMG\Documents\3. Client\Essential Energy\Probability of Failure Model\pof\pof\component.py", line 599, in expected_sensitivity
    self.mp_timeline(t_end=t_end, n_iterations=n_iterations)
  File "C:\Users\gtreseder\OneDrive - KPMG\Documents\3. Client\Essential Energy\Probability of Failure Model\pof\pof\component.py", line 446, in expected_risk_cost_df
    """ A wrapper for expected risk cost that returns a dataframe"""
  File "C:\Users\gtreseder\OneDrive - KPMG\Documents\3. Client\Essential Energy\Probability of Failure Model\pof\pof\component.py", line 542, in expected_risk_cost
    def expected_risk_cost(self):
  File "C:\Users\gtreseder\OneDrive - KPMG\Documents\3. Client\Essential Energy\Probability of Failure Model\pof\pof\component.py", line 542, in <dictcomp>
    def expected_risk_cost(self):
  File "C:\Users\gtreseder\OneDrive - KPM

KeyboardInterrupt: 

In [19]:
comp.plot_sens(
            var_id=var_id,
            y_axis='cost_cumulative'
        )

In [55]:
def expected_inspection_interval(x_min, x_max, n_iterations=100, t_end=100):
    """
    A simple method for returning the impact of changing an inspection interval. 
    TODO not needed now the method has been moved to the object expected sensitivities
    """
    rc = dict()
    comp.reset()

    for i in range(max(1, x_min),x_max,1):

        # Set t_interval
        for fm in comp.fm.values():
            if 'inspection' in list(fm.tasks):
                fm.tasks['inspection'].t_interval = i
        
        comp.mc_timeline(t_end=t_end, n_iterations=n_iterations)

        risk_cost = comp.expected_risk_cost_df().groupby(by=['task'])['cost'].sum()
        rc[i] = dict(risk_cost)
        rc[i]['inspection_interval'] = i

        # Reset component
        comp.reset()

    df = pd.DataFrame().from_dict(rc, orient='index')
    df['total_cost'] = df.drop(['inspection_interval', 'risk'], axis=1).sum(axis=1)
    df['total'] = df['total_cost'] + df['risk']
    
    return df

In [56]:
df = expected_inspection_interval(0, 10, 10)
clear_output()
df.head()

,risk,conditional_failure,functional_failure,inspection_groundline,inspection_protection,pole_saver_rod,termite_treatment,inspection_interval,total_cost,total
1,20000.0,700.0,2800.0,2305.0,0.0,40.0,10.0,1,5855.0,25855.0
2,15000.0,1400.0,2100.0,1910.0,0.0,30.0,20.0,2,5460.0,20460.0
3,20000.0,1050.0,2800.0,2180.0,0.0,20.0,20.0,3,6070.0,26070.0
4,20000.0,350.0,2800.0,2340.0,0.0,0.0,20.0,4,5510.0,25510.0
5,5000.0,1050.0,700.0,2730.0,0.0,10.0,20.0,5,4510.0,9510.0


## Populations - Consequence
We will consider the different consequence assets as a different 

In [14]:
filename = r"C:\Users\gtreseder\OneDrive - KPMG\Documents\3. Client\Essential Energy\Probability of Failure Model\inputs\csvs\ACS - Poles - Consequence Model Output.csv"
print("File Loading...")
df_cons = pd.read_csv(filename)
print("File Loaded")

File Loading...
File Loaded


Break the consequenes up into groups so they can be passed into the model

In [57]:
def get_bin_groups(df, bin_on='Total Consequence $', group_by='ASSET_ID', n_bins = 5):
    """ Groups an attribute"""

    df['bin_consequence'] = pd.qcut(df['Total Consequence $'], q=3, labels=['low', 'med', 'high'])

    groups = df.groupby(by='bin_consequence').count()

    return df

In [58]:
df_bins = get_bin_groups(df_cons)
df_bins.groupby(by='bin_consequence').count().head()

,Unnamed: 0,ASSET_ID,Depot (ZSG Area),Pole_LU_NSWMajo,Pole_LU_NSWDeta,Pole_LU_ALUMMaj,Pole_LU_ALUMDet,Road Crossing Conductor Type,HV_LV Status,BushfirePriority,...,C_Environment_Rank,Site_Bushfire_Moderate_Probability,Site_Bushfire_Severe_Probability,C_Bushfire_Dollars,Site_Environment_Insignificant_Probability,Site_Environment_Minor_Probability,C_Environment_Dollars,C_Financial_Dollars,Total Consequence $,Radial(Y/N)
bin_consequence,,,,,,,,,,,,,,,,,,,,,
low,459243,459243,459243,444182,444181,444182,444182,398918,398918,459243,...,459243,459243,459243,459243,459243,459243,459243,459243,459243,373
med,459243,459243,459243,457207,457202,457207,457207,436810,436810,459243,...,459243,459243,459243,459243,459243,459243,459243,459243,459243,446
high,459243,459243,459243,454364,454213,454364,454364,436925,436925,459243,...,459243,459243,459243,459243,459243,459243,459243,459243,459243,57316


# Grouping for impact
To understand the impact of different strategies at scale we will consider a depot lense

In [59]:
depots = ['Young Depot', 'Temora Depot', 'Yass Depot', 'Young Depot', 'Bathurst Depot']

In [60]:
index = 'Depot (ZSG Area)'
columns = 'bin_consequence' #'BushfirePriority'

consequence = df_bins.pivot_table(index=index, columns= columns, values = 'ASSET_ID', aggfunc=len).fillna(0).astype(int)
consequence.loc[depots].plot(kind='barh', title=f'{index} by {columns}');

Using 

In [61]:

def update_inspection_interval_figure():
    
    df = expected_inspection_interval(1,10, n_iterations=1000)
    df_plot = df.melt(id_vars='inspection_interval', var_name = 'source', value_name='cost')
    fig = px.line(df_plot, x="inspection_interval", y="cost", color='source', title='Risk v Cost at different Inspection Intervals')
    return fig

clear_output(True)
update_inspection_interval_figure().show()

100%|██████████| 1000/1000 [01:22<00:00, 12.15it/s]


In [11]:
def df_all_generator(by, col, sens_var, n_iterations=1000, t_min=1, t_max=5, t_end=100):
    """
    This function generates df_all
    """
    scen = dict()

    category_value = df_bins.groupby(by=by)[col].mean()

    for category, value in zip(category_value.index, category_value.to_numpy().astype(int)):
        comp = Component.demo()

        # Update consequence for all the failure modes
        for fm in comp.fm.values():
            fm.consequence.risk_cost_total = value

        scen[category] = expected_inspection_interval(t_min, t_max, n_iterations, t_end)

    df_all = pd.concat(scen).reset_index()
    df_all = df_all.rename(columns = {'level_0':by, 'level_1':sens_var})

    # Check there aren't any duplictes #TODO remove when changing to sensitivity
    df_all = df_all.loc[:,~df_all.columns.duplicated()]
    #TODO will need to get total time from components
    df_all['annual_value_lost'] = (df_all['total'] - df_all.groupby(by =by)['total'].transform('min'))/t_end # time considered

    # clear_output(True)
    return df_all

In [12]:
by = 'bin_consequence'
col = 'Total Consequence $'
sens_var = 'inspection_interval'

df_all = df_all_generator(by = by, col = col,
sens_var = sens_var)

df_all.head()

100%|██████████| 1000/1000 [00:14<00:00, 68.75it/s]


,bin_consequence,inspection_interval,inspection,on_condition_repair,on_condition_replacement,on_failure_replacement,risk,total_cost,total,annual_value_lost
0,low,1,9320,120,0,1010,5050,10450,15500,13.26
1,low,2,6202,112,0,1310,6550,7624,14174,0.00
2,low,3,4550,109,0,1700,8500,6359,14859,6.85
3,low,4,3530,104,0,2340,11700,5974,17674,35.00
4,med,1,9346,121,0,1070,5350,10537,15887,25.34


In [13]:

df_plot = df_all
fig = px.line(df_plot, x=sens_var, y="annual_value_lost", color=by, title=f'Risk v Cost at different {sens_var}', color_discrete_sequence=px.colors.qualitative.Vivid)
fig.show()

## Map to the population
Each asset can look up the model that most closely relates to it to get the value_lost for different strategies

In [14]:
df_pop = df_bins.groupby(by=['Depot (ZSG Area)', by])['ASSET_ID'].count().reset_index()
df_pop.head()

,Depot (ZSG Area),bin_consequence,ASSET_ID
0,Albury Depot,low,5350
1,Albury Depot,med,6560
2,Albury Depot,high,7497
3,Armidale Depot,low,7819
4,Armidale Depot,med,6628


In [15]:
y_var = 'annual_value_lost'
df_map = df_all[[by, sens_var, y_var]]
df_map.head()

,bin_consequence,inspection_interval,annual_value_lost
0,low,1,13.26
1,low,2,0.00
2,low,3,6.85
3,low,4,35.00
4,med,1,25.34


In [16]:
df_pop_map = df_pop.merge(df_map, on=by, how='outer')
df_pop_map.head()

,Depot (ZSG Area),bin_consequence,ASSET_ID,inspection_interval,annual_value_lost
0,Albury Depot,low,5350,1,13.26
1,Albury Depot,low,5350,2,0.00
2,Albury Depot,low,5350,3,6.85
3,Albury Depot,low,5350,4,35.00
4,Armidale Depot,low,7819,1,13.26


In [17]:
df_pop_map['total_value_lost'] = (df_pop_map.ASSET_ID * df_pop_map.annual_value_lost).where(df_pop_map.bin_consequence == 'high').fillna(0)
df_pop_map['total_value_lost'] = df_pop_map['total_value_lost'] + (df_pop_map.ASSET_ID * df_pop_map.annual_value_lost).where(df_pop_map.bin_consequence != 'high').fillna(0)
df_pop_map.head()

,Depot (ZSG Area),bin_consequence,ASSET_ID,inspection_interval,annual_value_lost,total_value_lost
0,Albury Depot,low,5350,1,13.26,70941.00
1,Albury Depot,low,5350,2,0.00,0.00
2,Albury Depot,low,5350,3,6.85,36647.50
3,Albury Depot,low,5350,4,35.00,187250.00
4,Armidale Depot,low,7819,1,13.26,103679.94


In [18]:
optimum_inspection = df_pop_map.groupby(by=['Depot (ZSG Area)', 'inspection_interval'])['total_value_lost'].sum().reset_index()

In [19]:
optimum_inspection = optimum_inspection.loc[optimum_inspection['Depot (ZSG Area)'].isin(depots)]

fig = px.line(
    optimum_inspection,
    x="inspection_interval",
    y="total_value_lost",
    color="Depot (ZSG Area)",
    title= 'Optimum Inspection Interval by Depot'
)
fig

In [20]:
optimum_inspection.groupby(by=['Depot (ZSG Area)']).min()

,inspection_interval,total_value_lost
Depot (ZSG Area),,
Bathurst Depot,1,0.0
Temora Depot,1,0.0
Yass Depot,1,0.0
Young Depot,1,0.0


To check that the sensitivity is behaving as expected

In [21]:
# comp.fm.pop('fast_aging')
# comp.fm.pop('slow_aging')
# comp.fm.pop('random')
# comp.fm

In [22]:
#only early_life
fig = figures.make_sensitivity_fig(comp, var_name = "comp-fm-early_life-tasks-inspection-cost", lower=0, upper=1000, step_size=100, n_iterations=10)
fig.show()

100%|██████████| 10/10 [00:00<00:00, 38.92it/s]


In [25]:
fig = figures.make_sensitivity_fig(comp, var_name = "comp-fm-fast_aging-dists-untreated-alpha", lower=0, upper=10, step_size=1, n_iterations=100)
fig.show()

100%|██████████| 100/100 [00:01<00:00, 93.28it/s]


In [24]:
# comp.fm["fast_aging"].tasks["inspection"].cost = 0
# comp.fm["fast_aging"].tasks["inspection"].cost

In [39]:
comp.expected_risk_cost()

{'fast_aging': {'risk': {'time': array([49, 53, 55, 61, 62, 63, 66, 67, 68, 69, 71, 72, 73, 77],
         dtype=int64),
   'cost': array([ 500.,  500.,  500.,  500., 2000.,  500., 1000., 1500., 1500.,
           500.,  500., 1000.,  500.,  500.]),
   'task_active': True,
   'fm_active': True},
  'inspection': {'time': array([ 10,  16,  22,  28,  34,  40,  46,  52,  58,  64,  70,  76,  82,
           88,  94, 100]),
   'cost': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
   'task_active': True,
   'fm_active': True},
  'on_condition_repair': {'time': array([35, 42, 47, 48, 53, 54, 55, 56, 59, 60, 61, 65, 66, 67, 71, 72]),
   'cost': array([ 1.,  1.,  3.,  2., 14.,  4.,  3.,  1., 17., 12.,  1., 29.,  9.,
           1., 10.,  2.]),
   'task_active': True,
   'fm_active': True},
  'on_condition_replacement': {'time': array([], dtype=float64),
   'cost': array([], dtype=float64),
   'task_active': True,
   'fm_active': True},
  'on_failure_replacement': {'time': 

In [40]:
comp.fm["fast_aging"].tasks["inspection"].cost = 1000
comp.fm["fast_aging"].tasks["inspection"].cost

1000

In [41]:
comp.mc_timeline(100)

100%|██████████| 100/100 [00:01<00:00, 90.15it/s]


In [42]:
comp.expected_risk_cost()

{'fast_aging': {'risk': {'time': array([54, 55, 60, 61, 64, 65, 67, 68, 69, 70, 72, 73, 77], dtype=int64),
   'cost': array([1000., 1000.,  500., 1000.,  500.,  500., 1000.,  500.,  500.,
          1500.,  500.,  500.,  500.]),
   'task_active': True,
   'fm_active': True},
  'inspection': {'time': array([ 10,  16,  22,  28,  34,  40,  46,  52,  58,  64,  70,  76,  82,
           88,  94, 100]),
   'cost': array([1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,  960.,
           930.,  870.,  820.,  810.,  810.,  810.,  810.]),
   'task_active': True,
   'fm_active': True},
  'on_condition_repair': {'time': array([47, 53, 54, 55, 59, 60, 61, 62, 65, 66, 67, 71, 72]),
   'cost': array([ 1.,  5.,  2.,  1., 26., 11.,  3.,  1., 27., 13.,  4., 14.,  3.]),
   'task_active': True,
   'fm_active': True},
  'on_condition_replacement': {'time': array([], dtype=float64),
   'cost': array([], dtype=float64),
   'task_active': True,
   'fm_active': True},
  'on_failure_replacement': {'time':

In [43]:
comp.fm["fast_aging"].timeline = {}
comp.fm["fast_aging"]._timelines = {}
comp.fm["fast_aging"].__dict__

{'_name': 'fast_aging',
 'units': 'years',
 'up_to_date': True,
 'dists': {'untreated': Distribution(name=untreated, alpha=50.0, beta=10.0, gamma=10.0,
  'init': Distribution(name=init, alpha=50.0, beta=10.0, gamma=10.0},
 'indicators': {'slow_degrading': <pof.indicator.ConditionIndicator at 0x2ba09c7c2b0>,
  'fast_degrading': <pof.indicator.ConditionIndicator at 0x2ba09d04bb0>},
 'conditions': {'slow_degrading': {'name': 'slow_degrading',
   'perfect': 100,
   'failed': 0,
   'pf_curve': 'linear',
   'pf_interval': 15,
   'pf_std': 0.5},
  'fast_degrading': {'name': 'fast_degrading',
   'perfect': 100,
   'failed': 0,
   'pf_curve': 'linear',
   'pf_interval': 8,
   'pf_std': 0.5}},
 'consequences': {},
 'tasks': {'inspection': <pof.task.Inspection at 0x2ba09da6760>,
  'on_condition_repair': <pof.task.ConditionTask at 0x2ba09da6640>,
  'on_condition_replacement': <pof.task.ConditionTask at 0x2ba09da6100>,
  'on_failure_replacement': <pof.task.ConditionTask at 0x2ba09da6340>},
 'init_s

In [26]:
df = comp.expected_sensitivity("comp-fm-early_life-tasks-inspection-cost", 50, 55)

100%|██████████| 100/100 [00:01<00:00, 79.42it/s]


In [37]:
from pof.failure_mode import FailureMode
import pof.demo as demo

fm = FailureMode.from_dict(demo.failure_mode_data["random"])
fm.name = "fm"

In [26]:
df = fm.sensitivity("fm-dists-untreated-alpha", 0, 5)

100%|██████████| 10/10 [00:00<00:00, 138.90it/s]


In [4]:
df

,alpha,direct_cost,risk_cost,total
1,1,1600,5000,6600
2,2,540,0,540
3,3,526,1666,2192
4,4,510,2500,3010


In [40]:
fig = make_sensitivity_fig(var_name = "fm-tasks-inspection-cost", lower=0, upper=1000, step=100, n_iterations=20)
fig.show()

100%|██████████| 20/20 [00:00<00:00, 188.67it/s]


In [7]:
failure_mode_data = dict(
        name='early_life',
        pf_curve = 'linear',
        pf_interval=10)

In [10]:
for var, val in failure_mode_data.items():
    d = {}
    d[var] = val
    print(d)

{'name': 'early_life'}
{'pf_curve': 'linear'}
{'pf_interval': 10}


In [3]:
comp.get_dash_ids()

['comp-active',
 'comp-fm-early_life-active',
 'comp-fm-early_life-pf_curve',
 'comp-fm-early_life-pf_interval',
 'comp-fm-early_life-pf_std',
 'comp-fm-early_life-dists-untreated-alpha',
 'comp-fm-early_life-dists-untreated-beta',
 'comp-fm-early_life-dists-untreated-gamma',
 'comp-fm-early_life-tasks-inspection-active',
 'comp-fm-early_life-tasks-inspection-p_effective',
 'comp-fm-early_life-tasks-inspection-cost',
 'comp-fm-early_life-tasks-inspection-t_interval',
 'comp-fm-early_life-tasks-inspection-t_delay',
 'comp-fm-early_life-tasks-inspection-trigger-condition-instant-lower',
 'comp-fm-early_life-tasks-inspection-trigger-condition-instant-upper',
 'comp-fm-early_life-tasks-inspection-trigger-state-initiation',
 'comp-fm-early_life-tasks-inspection-impact-state-detection',
 'comp-fm-early_life-tasks-on_condition_replacement-active',
 'comp-fm-early_life-tasks-on_condition_replacement-p_effective',
 'comp-fm-early_life-tasks-on_condition_replacement-cost',
 'comp-fm-early_life-t

In [3]:
def component_val_to_csv():
    comp = Component().demo()

    
    d = {}
    d_split = {}
    for id_ in comp.get_dash_ids():
        
        id_str = id_.split("comp" + "-", 1)[1]
        id_str = id_str.split("-")

        value = comp

        for s in id_str:
            if s == "fm":
                value = value.fm
            elif s == "active":
                value = value.active
            elif s == "pf_curve":
                value = value.pf_curve
            elif s == "pf_interval":
                value = value.pf_interval
            elif s == "pf_std":
                value = value.pf_std
            elif s == "dists":
                value = value.dists
            elif s == "alpha":
                value = value.alpha
            elif s == "beta":
                value = value.beta
            elif s == "gamma":
                value = value.gamma
            elif s == "tasks":
                value = value.tasks
            elif s == "p_effective":
                value = value.p_effective
            elif s == "cost":
                value = value.cost
            elif s == "t_interval":
                value = value.t_interval
            elif s == "t_delay":
                value = value.t_delay
            elif s == "states":
                value = value.states
            elif s == "impact":
                value = value.impacts
            elif s == "trigger":
                value = value.triggers
            elif s == id_str[-1]:
                # if isinstance(value[s], str):
                #     continue
                # elif isinstance(value[s], bool):
                #     continue
                # else:
                #     d[id_] = value[s]
                #     d_split[id_] = id_.split("-")
                d[id_] = value[s]
                d_split[id_] = id_.split("-")
            else:
                value = value[s]
    df = pd.DataFrame(d.items(), columns=['name', 'mean'])
    test = pd.DataFrame(d_split.items())
    df[["key_1", "key_2", "key_3", "key_4", "key_5", "key_6", "key_7", "key_8", "key_9"]] = pd.DataFrame(test[1].to_list())
    df.set_index(["name"], drop = True).to_csv("csv.csv")


In [12]:
# import pof.test.fixtures
# comp = Component.from_dict(fixtures.component_data_slow["comp"])

comp = Component().demo()
# comp.fm.pop('fast_aging')
# comp.fm.pop('early_life')
# comp.fm.pop('random')

d = {}
d_split = {}
for id_ in comp.get_dash_ids():
    
    id_str = id_.split("comp" + "-", 1)[1]
    id_str = id_str.split("-")

    value = comp

    for s in id_str:
        if s == "fm":
            value = value.fm
        elif s == "active":
            value = value.active
            if isinstance(value, str):
                continue
            elif isinstance(value, bool):
                continue
            else:
                d[id_] = value
                d_split[id_] = id_.split("-")
        elif s == "pf_curve":
            value = value.pf_curve
            if isinstance(value, str):
                continue
            elif isinstance(value, bool):
                continue
            else:
                d[id_] = value
                d_split[id_] = id_.split("-")
        elif s == "pf_interval":
            value = value.pf_interval
            if isinstance(value, str):
                continue
            elif isinstance(value, bool):
                continue
            else:
                d[id_] = value
                d_split[id_] = id_.split("-")
        elif s == "pf_std":
            value = value.pf_std
            if isinstance(value, str):
                continue
            elif isinstance(value, bool):
                continue
            else:
                d[id_] = value
                d_split[id_] = id_.split("-")
        elif s == "dists":
            value = value.dists
        elif s == "alpha":
            value = value.alpha
            if isinstance(value, str):
                continue
            elif isinstance(value, bool):
                continue
            else:
                d[id_] = value
                d_split[id_] = id_.split("-")
        elif s == "beta":
            value = value.beta
            if isinstance(value, str):
                continue
            elif isinstance(value, bool):
                continue
            else:
                d[id_] = value
                d_split[id_] = id_.split("-")
        elif s == "gamma":
            value = value.gamma
            if isinstance(value, str):
                continue
            elif isinstance(value, bool):
                continue
            else:
                d[id_] = value
                d_split[id_] = id_.split("-")
        elif s == "tasks":
            value = value.tasks
        elif s == "p_effective":
            value = value.p_effective
            if isinstance(value, str):
                continue
            elif isinstance(value, bool):
                continue
            else:
                d[id_] = value
                d_split[id_] = id_.split("-")
        elif s == "cost":
            value = value.cost
            if isinstance(value, str):
                continue
            elif isinstance(value, bool):
                continue
            else:
                d[id_] = value
                d_split[id_] = id_.split("-")
        elif s == "t_interval":
            value = value.t_interval
            if isinstance(value, str):
                continue
            elif isinstance(value, bool):
                continue
            else:
                d[id_] = value
                d_split[id_] = id_.split("-")
        elif s == "t_delay":
            value = value.t_delay
            if isinstance(value, str):
                continue
            elif isinstance(value, bool):
                continue
            else:
                d[id_] = value
                d_split[id_] = id_.split("-")
        elif s == "states":
            value = value.states
        elif s == "impact":
            value = value.impacts
        elif s == "trigger":
            value = value.triggers
        elif s == id_str[-1]:
            if isinstance(value[s], str):
                continue
            elif isinstance(value[s], bool):
                continue
            if isinstance(value[s], list):
                continue
            else:
                d[id_] = value[s]
                d_split[id_] = id_.split("-")
        else:
            value = value[s]
df = pd.DataFrame(d.items(), columns=['name', 'mean'])
df["sd"] = df['mean']*.2
df


,name,mean,sd
0,comp-fm-early_life-pf_interval,10.0,2.0
1,comp-fm-early_life-pf_std,0.0,0.0
2,comp-fm-early_life-dists-untreated-alpha,1000000.0,200000.0
3,comp-fm-early_life-dists-untreated-beta,0.5,0.1
4,comp-fm-early_life-dists-untreated-gamma,0.0,0.0
...,...,...,...
79,comp-fm-fast_aging-tasks-on_condition_replacem...,0.0,0.0
80,comp-fm-fast_aging-tasks-on_condition_replacem...,20.0,4.0
81,comp-fm-fast_aging-tasks-on_condition_replacem...,0.0,0.0
82,comp-fm-fast_aging-tasks-on_condition_replacem...,20.0,4.0


In [3]:
df_inspection_slow = df[df['name'].str.contains('cost')]
df_inspection_slow = df_inspection_slow[df_inspection_slow['name'].str.contains('slow')]
df_inspection_slow

,name,mean,sd
38,comp-fm-slow_aging-tasks-inspection-cost,55,5
47,comp-fm-slow_aging-tasks-on_condition_repair-cost,100,5
56,comp-fm-slow_aging-tasks-on_condition_replacem...,5000,5


In [4]:
df_inspection_slow['sd'] = df_inspection_slow['mean']*.2
df_inspection_slow

,name,mean,sd
38,comp-fm-slow_aging-tasks-inspection-cost,55,11
47,comp-fm-slow_aging-tasks-on_condition_repair-cost,100,20
56,comp-fm-slow_aging-tasks-on_condition_replacem...,5000,1000


In [5]:
conf = 0.9
n_increments = 10
n_iterations = 100
#if it's even add one

df_sens = comp.sensitivity(var_name = df_inspection_slow, conf =  conf, n_increments = n_increments, n_iterations = n_iterations)

comp.make_sensitivity_plot(data = df_sens, x_axis = 'conf', y_axis = 'var_name', failure_mode = 'slow_aging', plot_type = 'heatmap', z_axis = 'risk_cost')


100%|██████████| 100/100 [00:03<00:00, 31.13it/s]
